In [131]:
from PIL import Image
import matplotlib.pyplot as pyplot
import numpy as np
import random
import glob
import sys

In [67]:
SIZE = 512
PATCH = 64

In [90]:
def all_transpositions(im):
    yield im
    yield im.transpose(Image.FLIP_LEFT_RIGHT)
    yield im.transpose(Image.FLIP_TOP_BOTTOM)
    yield im.transpose(Image.ROTATE_180)

In [128]:
def random_patch(im):
    i = random.randrange(0, SIZE, PATCH)
    j = random.randrange(0, SIZE, PATCH)
    return im.crop((i, j, i + PATCH, j + PATCH))

def generate_patches(im):
    total = 0
    for i in range(0, SIZE, 2 * PATCH):
        for j in range(0, SIZE, PATCH):
            patch = im.crop((i, j, i + 2 * PATCH, j + PATCH))
            for image in all_transpositions(patch):
                yield image, 1
                total += 1
    for i in range(0, SIZE, PATCH):
        for j in range(0, SIZE, 2 * PATCH):
            patch = im.crop((i, j, i + PATCH, j + 2 * PATCH))
            for image in all_transpositions(patch.transpose(Image.ROTATE_90)):
                yield image, 1
                total += 1
    for _ in range(total):
        a = random_patch(im)
        b = random_patch(im)
        image = Image.new('L', (2 * PATCH, PATCH))
        image.paste(a, (0, 0))
        image.paste(b, (PATCH, 0))
        yield image, 0

In [133]:
xs, ys = [], []
for path in glob.glob('*.png'):
    im = Image.open(path).convert('L')
    assert im.height == SIZE
    assert im.width == SIZE
    for x, y in generate_patches(im):
        xs.append(np.asarray(x).flatten() / 255)
        ys.append(y)
print(len(xs), len(ys))
print(sys.getsizeof(xs), sys.getsizeof(ys))

307200 307200
2678096 2678096
